### **Training** ###
This notebook trains the YOLOv8 segmentation model to detect a new rooftop class. We considered the available YOLO configurations—nano, small, medium, large, and xl. While the larger sizes generally yield better results, they come with higher computational demands. To balance performance and computational expense, we chose the small YOLO model.

The training datasets were derived from the RAMP dataset and prepared using the preprocessing notebook. To optimize the model results, we trained YOLO using data version 1, 2, and 3. Each data version was successively pruned (V3, V2, .. V0), meaning V3 was pruned from V2 and so on to V0 so that each version contains the prunings from all of the previous.

RAMP-DATA-V0: 'Full Dataset' is 100k image-label pairs converted from RAMP's TIF-GEOJSON formats within 22 regional folders and stored in train-val-test folders with a 70-15-15 split; zipped at 15.2 GB downloaded from Source Cooperative.

YOLO-DATA-V1: 'Preprocessing' was a process that resulted in 83k image-label pairs in JPG-TXT formats from 20 regions, zipped at 3.5 GB. We pruned 17k images from V0, reducing the background images from 18% to 5%. Shanghai and Paris regions were removed due to image format issues. A minimal manual review discovered a few labeling issues (n=38), ensuring the highest data quality.

YOLO-DATA-V2: 'Pruning #2' resulted in 79k image-label pairs in JPG-TXT formats from 20 regions, zipped at 3.3 GB. We scored and analyzed inference results from the YOLO model trained on DATA-V1. The analysis revealed double the expected number of left-tail outliers. To improve labeling accuracy, we removed 4k outliers, assuming that most of these labels were errors.

YOLO-DATA-V3: 'Zoom 19' resulted in 55k image-label pairs in JPG-TXT formats from 20 regions, zipped at 2.4 GB. We extracted the x and y scales from the TIF files and found the range to be between 0.3 and 0.52 meters per pixel, with 70% between 0.3 and 0.4. To test the effect of training near Zoom 19, which equates to 0.3 meters per pixel, we removed all image labels with scales greater than 0.4.

To relieve the time constraints, we recommend training with a GPU. Google Colab's High-RAM T4 can train 100 epochs in 15 hours using a large batch size and cache enabled to minimize disk I/O. Using a 2023 laptop with a CPU, training YOLO for 100 epochs takes about 2.4 weeks.

In [ ]:
!pip3 install -q ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.0/869.0 kB 40.1 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
import os
import yaml

In [ ]:
data_choice = input("Which dataset do you want to download? Enter 'v1', 'v2', or 'v3': ")

# Download and unzip the specified data version
if data_choice.lower() == 'v1':
    print("\nDownloading and unzipping v1...")
    !gdown --fuzzy https://drive.google.com/file/d/10XGskgMGdkXTqbGDGu9BRtJklq_lfFTd/view?usp=sharing
    !unzip -q data1*.zip
elif data_choice.lower() == 'v2':
    print("\nDownloading and unzipping v2...")
    !gdown --fuzzy https://drive.google.com/file/d/1zXwvKLxzzOBp1bQ67nqfOgM55BI6sHho/view?usp=sharing
    !unzip -q data2*.zip
elif data_choice.lower() == 'v3':
    print("\nDownloading and unzipping v3...")
    !gdown --fuzzy https://drive.google.com/file/d/1K-jaOylXjukvCC6Uu8Ms2e7LxfL0-qWg/view?usp=sharing
    !unzip -q data3*.zip
else:
    print("Invalid input.")

# Download and unzip models if not found.
if not os.path.exists('yolov8s-seg.pt'):
    print('\nDownloading and unzipping YOLOv8 models...')
    !gdown --fuzzy https://drive.google.com/file/d/189U5fTomw3-QPjPpkppiXVSKa-sVIQIl/view?usp=sharing
    print('Unzipping models...')
    !unzip -q yolov8_-seg.zip
    print('Models downloaded')
else:
    print('\nModels already downloaded')

# Download the training arguments
print('\nDownloading training arguments...')
!gdown --fuzzy https://drive.google.com/file/d/13KLv4ywEVNVQHWhahIapSDDEzJWcAmT_/view?usp=sharing
!gdown --fuzzy https://drive.google.com/file/d/1_N8H6akiKqFwezDjqfjVuXDvuP6zJZGP/view?usp=sharing

Which dataset do you want to download? Enter 'v1', 'v2', or 'v3': v2

Downloading...
From (original): https://drive.google.com/uc?id=1zXwvKLxzzOBp1bQ67nqfOgM55BI6sHho
From (redirected): https://drive.google.com/uc?id=1zXwvKLxzzOBp1bQ67nqfOgM55BI6sHho&confirm=t&uuid=9922bce6-fd58-4984-a879-48a9613e492a
To: /content/data2_ramp_data_yolo_iou.zip
100% 3.27G/3.27G [00:57<00:00, 56.5MB/s]

Downloading...
From (original): https://drive.google.com/uc?id=189U5fTomw3-QPjPpkppiXVSKa-sVIQIl
From (redirected): https://drive.google.com/uc?id=189U5fTomw3-QPjPpkppiXVSKa-sVIQIl&confirm=t&uuid=6d1b432a-06de-4365-85b8-0bbbb940bb25
To: /content/yolov8_-seg.zip
100% 298M/298M [00:07<00:00, 39.7MB/s]
Unzipping models...
Models downloaded

Downloading...
From: https://drive.google.com/uc?id=13KLv4ywEVNVQHWhahIapSDDEzJWcAmT_
To: /content/args_500e.yaml
100% 504/504 [00:00<00:00, 3.44MB/s]
Downloading...
From: https://drive.google.com/uc?id=1_N8H6akiKqFwezDjqfjVuXDvuP6zJZGP
To: /content/args_500e_tuned.yaml
10

In [ ]:
# Write the YAML file for YOLO that contains the class names and train/val paths
YAML = 'data.yaml'

# Set the path based on the version of the dataset
if data_choice == 'v1':
    path = f'{os.getcwd()}/ramp_data_yolo'
elif data_choice == 'v2':
    path = f'{os.getcwd()}/ramp_data_yolo_iou'
elif data_choice == 'v3':
    path = f'{os.getcwd()}/ramp_data_yolo_iou_scale'

# Set the attribute for the YAML file
attr = {
    'path': path,
    'train': 'train/images',
    'val': 'val/images',
    'names': {
        0: 'rooftop'
    }
}

# Print the path
print(f'Writing the data file with path={path}')

# Write the file
with open(YAML, 'w') as f:
    yaml.dump(attr, f)

Writing the data file with path=/content/ramp_data_yolo_iou


In [ ]:
# HIGHLY RECOMMENDED if your training is of a long duration
# Connect to Google Drive so that if session is terminated you can resume
from google.colab import drive

# Uncomment if needed
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# For testing purposes the fraction argument can be used to train on a subset of the data
FRACTION = 1.0

# Setting the starting model
MODEL = 'yolov8s-seg.pt'

# Setting the project name
PROJECT = '/content/drive/MyDrive/Omdena'

# Load the arguments from YAML
ARGS_FN = 'args_500e_tuned.yaml'
with open(ARGS_FN, 'r') as f:
    args = yaml.safe_load(f)

# Generate a name for the training session
name = f'{MODEL}_data_{data_choice}'
name = name.replace('.pt', '')

# Prompt the user for input
choice = input("Do you want to start or resume training? Enter 'new' for new training or 'resume' to resume training: ")

# Process the user input
if choice.lower() == 'new':
    print("\nStarting new training...\n")

    # Load the model
    model = YOLO(MODEL)

    # Train the model
    results = model.train(data=YAML, fraction=FRACTION, project=PROJECT, name=name, **args)

elif choice.lower() == 'resume':
    print("Resuming training...")

    # Load the model
    modelwpath = f'{PROJECT}/{name}/weights/last.pt'
    print(f'Loading model from {modelwpath}')
    model = YOLO(modelwpath)

    # Resume training
    results = model.train(resume=True)

else:
    print("Invalid input.")

Resuming training...
Loading model from /content/drive/MyDrive/Omdena/yolov8s-seg_data_v2/weights/last.pt
Ultralytics YOLOv8.2.78 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=/content/drive/MyDrive/Omdena/yolov8s-seg_data_v2/weights/last.pt, data=data.yaml, epochs=500, time=None, patience=500, batch=64, imgsz=256, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive/Omdena, name=yolov8s-seg_data_v2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=/content/drive/MyDrive/Omdena/yolov8s-seg_data_v2/weights/last.pt, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=False, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stri

100%|██████████| 755k/755k [00:00<00:00, 99.6MB/s]


TensorBoard: Start with 'tensorboard --logdir /content/drive/MyDrive/Omdena/yolov8s-seg_data_v2', view at http://localhost:6006/

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f  

100%|██████████| 6.25M/6.25M [00:00<00:00, 222MB/s]


AMP: checks passed ✅


...

                   all      11907     189826      0.765      0.688      0.751      0.477      0.759       0.67      0.737      0.415

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


    500/500      4.61G     0.9682      1.369     0.8446     0.9189         28        256: 100%|██████████| 870/870 [08:40<00:00,  1.67it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 94/94 [01:25<00:00,  1.10it/s]


                   all      11907     189826      0.765      0.687      0.751      0.477      0.759      0.671      0.737      0.414

78 epochs completed in 13.050 hours.
Optimizer stripped from /content/drive/MyDrive/Omdena/yolov8s-seg_data_v2/weights/last.pt, 23.9MB
Optimizer stripped from /content/drive/MyDrive/Omdena/yolov8s-seg_data_v2/weights/best.pt, 23.8MB

Validating /content/drive/MyDrive/Omdena/yolov8s-seg_data_v2/weights/best.pt...
Ultralytics YOLOv8.2.78 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s-seg summary (fused): 195 layers, 11,779,987 parameters, 0 gradients, 42.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 94/94 [01:44<00:00,  1.11s/it]


                   all      11907     189826      0.775      0.703      0.784      0.509      0.767      0.686      0.768      0.441
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /content/drive/MyDrive/Omdena/yolov8s-seg_data_v2
